In [1]:
from utils import *
from data_processing import *
today = datetime.date.today()


In [2]:
config = TsConf()
df = TsDataFrame(pd.read_csv(config.cmapss_url))
df.columns = config.columns_old
df = df[config.columns_new]

In [3]:
df = df.cluster(config)

In [4]:
df = df.data_norm(config)

In [5]:
config.ewm = 20
df = df.ewma(config)

In [6]:
train = df[(["FILEID","ENGINEID", "TIMECYCLE"]+config.select_feat+["RUL"])][df.FILEID.isin(config.train_fids)]
test = df[(["FILEID","ENGINEID", "TIMECYCLE"]+config.select_feat+["RUL"])][df.FILEID.isin(config.test_fids)]

In [7]:
train.tail()

,FILEID,ENGINEID,TIMECYCLE,Total temp at LPC out (T24),Total temp at HPC out (T30),Total temp at LPT out (T50),Physical core speed (Nc),Static pres at HPC out (Ps30),Corrected core speed (NRc),Bypass Ratio (BPR),Bleed Enthalpy (htBleed),RUL
87480,102,260,312,1.181564,1.400266,1.404234,4.871879,1.204583,4.903128,1.136318,1.453609,4
87481,102,260,313,1.215280,1.434809,1.409812,4.980184,1.258923,4.997293,1.199401,1.521893,3
87482,102,260,314,1.239175,1.488724,1.395823,5.076812,1.276316,5.098160,1.309503,1.547101,2
87483,102,260,315,1.309800,1.510402,1.441107,5.163097,1.333994,5.198679,1.401138,1.607943,1
87484,102,260,316,1.338534,1.505439,1.462482,5.260137,1.377636,5.300208,1.418648,1.659060,0


# Model training

## LSTM (Sibur)

In [8]:
n_eng_train = max(train.ENGINEID)
n_eng_test = max(test.ENGINEID)

In [9]:
train.target = config.target
train.groupids = config.groupids
train.timestamp = config.timestamp
test.target = config.target
test.groupids = config.groupids
test.timestamp = config.timestamp

In [10]:
Train = True
Predict = True
mode = "Train"

In [11]:
config.sequence_length = 30
config.batch_size = 30
config.shift = 30
config.restore_model = True
config.random = False

In [12]:
config.display()


Configurations:
alpha                          0
ann_hidden                     16
batch_size                     30
cmapss_n_clusters              6
cmapss_url                     input/dataset.csv
columns_new                    ['FILEID', 'ENGINEID', 'TIMECYCLE', 'Alt', 'Mach', 'TRA', 'Total temp at fan in (T2)', 'Total temp at LPC out (T24)', 'Total temp at HPC out (T30)', 'Total temp at LPT out (T50)', 'Pres at fan in (P2)', 'Total pres in bypass-duct (P15)', 'Total pres at HPC out (P30)', 'Physical fan speed (Nf)', 'Physical core speed (Nc)', 'Engine pres ratio (epr=P50/P2)', 'Static pres at HPC out (Ps30)', 'Ratio of fuel flow to Ps30 (phi)', 'Corrected fan speed (NRf)', 'Corrected core speed (NRc)', 'Bypass Ratio (BPR)', 'Burner fuel-air ratio (farB)', 'Bleed Enthalpy (htBleed)', 'Demanded fan speed (Nf_dmd)', 'Demanded corrected fan speed (PCNfR_dmd)', 'HPT coolant bleed (W31)', 'LPT coolant bleed (W32)', 'RUL']
columns_old                    ['ENGINEID', 'TIMECYCLE', 'Alt', '

In [13]:
m = TsLSTM(config)

Building LSTM...
My Conv Shape: [None, 30, 8]


In [14]:
m.train(train, test, config)

In [15]:
config.shift=1

In [27]:
y_pred = m.predict(train[train.ENGINEID==2], config)

INFO:tensorflow:Restoring parameters from ./save/save_lstm/lstm_2_layers
Model restored from file: ./save/save_lstm/lstm_2_layers
Prediction for submit...
#of validation points: 269 #datapoints covers from minibatch: 900


In [28]:
#out = y_pred.reshape(int(len(y_pred)/config.sequence_length),config.sequence_length)

In [29]:
len(y_pred)

182

In [30]:
y_pred

array([113.60386 , 113.63176 , 113.55859 , 113.34731 , 113.219666,
       113.19232 , 113.25738 , 113.227974, 113.36677 , 113.485916,
       113.58614 , 113.646675, 113.703926, 113.73731 , 113.751076,
       113.78756 , 113.80817 , 113.83243 , 113.85174 , 113.867615,
       113.85786 , 113.86784 , 113.860435, 113.79538 , 113.72418 ,
       113.64419 , 113.45429 , 113.214516, 112.96029 , 112.58002 ,
       112.56138 , 112.81535 , 112.82387 , 113.06267 , 113.16201 ,
       113.18119 , 113.26077 , 113.25008 , 113.316216, 113.40354 ,
       113.44228 , 113.44799 , 113.444046, 113.41739 , 113.17888 ,
       113.01974 , 112.90731 , 112.84741 , 112.75669 , 112.55583 ,
       112.146416, 111.415344, 110.81298 , 110.28538 , 110.31758 ,
       110.62182 , 110.08656 , 110.16248 , 109.79988 , 110.30214 ,
       111.789764, 112.63298 , 113.173584, 113.814865, 114.09719 ,
       114.20879 , 114.29353 , 114.2685  , 114.24787 , 114.2387  ,
       114.17618 , 113.975494, 113.78601 , 113.43162 , 113.390

In [20]:
#pd.DataFrame(out)

In [21]:
#y_submit = full_prediction*std_y+mean_y
#sub_file = pd.DataFrame()
#sub_file["timestamp"] = submit_X.index[-2872:]
#sub_file["target"] = y_submit[-2872:]
#sub_file.to_csv('submit2.csv', index=False)

## Gradient Boosting Regression

### TS fresh dataset

In [22]:
#add_lag_roll(test.iloc[:,:-2], feat_names, lag=lag, step=step, roll=0).head()

In [23]:
#df.groupby(["FILEID", "ENGINEID"])

In [24]:
#X=pd.DataFrame()
#y=list()

In [25]:
#for name, group in df.groupby(["FILEID", "ENGINEID"]):
    #X = pd.concat([X, add_lag_roll(group[feat_names], feat_names, lag=lag, step=step)], axis=0)
    #y = y.append(group["RUL"][lag:])
    
    

In [26]:
#X.tail(1).values